In [29]:
from scrapy.selector import Selector
import requests
import openpyxl
from openpyxl import load_workbook
import pandas as pd
import datetime





#--------> Note <-------#

#Create the a excel file with below file name here it is urls.xlsx with Fields "Urls" and "Page Numbers"


# Here you have to give the page numbers below. I gave default 10 
sheet_flag = False
# format the date and time to include seconds
timestamp = time.time()

workbook_name ="data"+str(int(timestamp)) +'.xlsx'


file_name = "urls.xlsx"

df = pd.read_excel(file_name)
urls = list(df['Urls'])
page_nums = list(df['Page Numbers'])

for j in range(len(urls)):
    print(urls[j])
    print(page_nums[j])

    for i in range(1,page_nums[j]):
        print("page ---> ",i)

        # Here you have to change the url for which you need to scrape. You have to change part of the url before "?page="
        url = urls[j] +"?page="+str(i)
        print("url-->",url)

        response = requests.get(url)
        print(response)

        # Create a Selector from the response text
        res = Selector(text=response.text)

        # Extract raw data using CSS selectors
        raw_data = res.css('div[data-section-id="template--15367256998119__main"] div[class="block-inner-inner"]')
        data_dict = {'ProductTitle':[],'Price':[],'MRP':[], 'Colour':[], 'URL':[], 'Product_id':[], 'Size':[], 'Category':[]}
    #     print(raw_data)
        # Loop through the raw data and print the product titles
        for data in raw_data:
#             print("in data ")
            product_title = data.css('div[class="product-info"] div[class="product-block__title"] h4::text' ).extract_first()
            price= data.css('div[class="product-info"] div[class="product-price"] span[class="product-price__item product-price__amount product-price__amount--on-sale theme-money"] ::text').extract_first()
            Mrp_Price=data.css('div[class="product-info"] div [class="product-price__item product-price__compare theme-money"]::text').extract_first()
            colour= data.css('div[class="product-info"] div [class="product-block-options__item__text"]::text').extract_first()
            size1= data.css('div[class="product-info"] div [class="product-block-options__item__text"]::text').extract()
            size11 = size1[1:]
            size=str(size11)

            link= data.css('div [class="product-link"]::attr(href)').extract_first()
            try:
                Url= url.split('?')[0]+link
            except:
                Url = '-'
            product_id=data.css('::attr(data-id)').extract_first()

            try:
                data_dict['ProductTitle'].append(product_title.strip())
            except:
                data_dict['ProductTitle'].append('-')
            data_dict['Price'].append(price)
            data_dict['MRP'].append(Mrp_Price)
            data_dict['Colour'].append(colour)
            data_dict['URL'].append(Url)
            data_dict['Product_id'].append(product_id)
            data_dict['Size'].append(size)

            try:
                category = ' '.join(url.split('/')[-1].split('?')[0].split('-'))
            except:
                category = '-'

            data_dict['Category'].append(category)
            
            

            # get the current date and time

        

        data_df = pd.DataFrame(data_dict)
        columns = data_df.columns.to_list()    
        df_values = data_df.values.tolist()
        df_values.insert(0,columns)

        print("sheet_flag",sheet_flag)
        if sheet_flag == False:

            sheet_flag = True
            wb = openpyxl.Workbook()
            ws = wb.active
            for row in df_values:
                ws.append(row)
                wb.save(workbook_name)

            data_dict = {'ProductTitle':[],'Price':[],'MRP':[], 'Colour':[], 'URL':[], 'Product_id':[], 'Size':[],'Category':[]}


        else:
            df_values = data_df.values.tolist()
            wb = load_workbook(workbook_name)
            page = wb.active
            for row in df_values:
                #print(row)
                page.append(row)
                wb.save(filename=workbook_name)
            data_dict = {'ProductTitle':[],'Price':[],'MRP':[], 'Colour':[], 'URL':[], 'Product_id':[], 'Size':[],'Category':[]}





https://www.campusshoes.com/collections/mens-sneakers
2
page --->  1
url--> https://www.campusshoes.com/collections/mens-sneakers?page=1
<Response [200]>
sheet_flag False
https://www.campusshoes.com/collections/mens-sports-shoes
4
page --->  1
url--> https://www.campusshoes.com/collections/mens-sports-shoes?page=1
<Response [200]>
sheet_flag True
page --->  2
url--> https://www.campusshoes.com/collections/mens-sports-shoes?page=2
<Response [200]>
sheet_flag True
page --->  3
url--> https://www.campusshoes.com/collections/mens-sports-shoes?page=3
<Response [200]>
sheet_flag True
